# Data Processing

In [ ]:
from collections import Counter
import string

import pandas as pd

from src import data_processing

In [ ]:
MIN_YEAR = 2011
MAX_YEAR = 2020

PUNCTUATION = set(string.punctuation)

## Reactor -> Power Generated

In [ ]:
reactors_df = pd.read_excel('./data/raw/reactors-operating.xlsx')
reactors_df = reactors_df[[
    'Plant Name',
    'Capacity MWe',
    '2020 Capacity Factor (Percent)',
    '2019 Capacity Factor (Percent)',
    '2018 Capacity Factor (Percent)',
    '2017 Capacity Factor (Percent)',
    '2016 Capacity Factor (Percent)',
    '2015 Capacity Factor (Percent)',
    '2014 Capacity Factor (Percent)',
    '2013 Capacity Factor (Percent)',
    '2012 Capacity Factor (Percent)',
    '2011 Capacity Factor (Percent)'
]]
reactors_df.head()

In [ ]:
reactors_melt_df = pd.melt(reactors_df, id_vars=['Plant Name', 'Capacity MWe'])
reactors_melt_df = reactors_melt_df.rename(
    columns={
        'Capacity MWe': 'total capacity (MWe)',
        'Plant Name': 'name',
        'value': 'percent of capacity',
        'variable': 'year',
    }
)
reactors_melt_df

In [ ]:
reactors_melt_df['year'] = reactors_melt_df['year'].str.replace(' Capacity Factor \(Percent\)', '').astype(int)

reactors_melt_df['production (MWe)'] = reactors_melt_df['total capacity (MWe)'] * reactors_melt_df['percent of capacity']
reactors_melt_df['production (MWe)'] = reactors_melt_df['production (MWe)'].fillna(0.0)
reactors_melt_df = reactors_melt_df[['year', 'name', 'production (MWe)']]
reactors_melt_df

## Reactor -> Number of Scrams

In [ ]:
scram_df = pd.read_excel('./data/raw/scram-event-notifications.xlsx')
scram_df = scram_df[scram_df['NRC_retract_flag'] == 0]
scram_df = scram_df[['Year ','Plant Name', 'Scram #']]
scram_df.head()

In [ ]:
scram_agg_df = scram_df.groupby(['Year ', 'Plant Name'], as_index=False).count()
scram_agg_df = scram_agg_df.rename(
    columns={'Year ': 'year', 'Plant Name': 'name', 'Scram #': 'scrams'}
)
scram_agg_df.head()

## Combine Data Sets

In [ ]:
words = []
for entry in reactors_df['Plant Name'].tolist():
    entry_nopunct = ''.join(ch for ch in entry if ch not in PUNCTUATION)
    words.extend(entry_nopunct.upper().split())

word_counter = Counter(words)
common_words = [w for w, c in word_counter.items() if c > 10]
common_words += ['NO']
common_words

In [ ]:
reactors_melt_df['name'] = reactors_melt_df['name'].apply(data_processing.split_name_fn(common_words))

reactors_melt_df = reactors_melt_df[
    (reactors_melt_df['year'] >= MIN_YEAR)
    & (reactors_melt_df['year'] <= MAX_YEAR)
]
print(len(reactors_melt_df))
assert(reactors_melt_df.set_index(['name','year']).index.is_unique)
reactors_melt_df.head()

In [ ]:
scram_agg_df['name'] = scram_agg_df['name'].apply(data_processing.split_name_fn(common_words))

scram_agg_df = scram_agg_df[
    (scram_agg_df['year'] >= MIN_YEAR)
    & (scram_agg_df['year'] <= MAX_YEAR)
]
print(len(scram_agg_df))
assert(scram_agg_df.set_index(['name','year']).index.is_unique)
scram_agg_df.head()

In [ ]:
merged_df = pd.merge(reactors_melt_df, scram_agg_df, how='outer', on=['name', 'year'])
merged_df.head()

In [ ]:
# Plants with scrams, but no production information
merged_df[merged_df['production (MWe)'].isna()]['name'].unique().tolist()

In [ ]:
merged_df = merged_df[~merged_df['production (MWe)'].isna()]

merged_df['scrams'] = merged_df['scrams'].fillna(0).astype(int)
merged_df = merged_df[merged_df['production (MWe)'] > 0]  # Only active plant/years
merged_df = merged_df.sort_values(['year', 'name', 'production (MWe)', 'scrams'])

print(len(merged_df))
merged_df.to_pickle('./data/processed/counts_df.pkl')
merged_df.head()